In [1]:
import json
import time
import itertools
from torch.utils.data import Dataset
import ipdb
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from pycocotools.coco import COCO
from skimage.morphology import disk, square
from collections import defaultdict
import scipy
import config
selem = disk(2)
square_dil = square(5)

In [ ]:
cosider_flip_too = 0
all_10_ans_common=1

coco_val_pkl = './../iv_vqa_generation/coco_areas_and_intersection/coco_vqa_val2014.json'
coco_train_pkl = './../iv_vqa_generation/coco_areas_and_intersection/coco_vqa_train2014.json'

COCO_val = COCO(config.coco_ann_dir + 'instances_val2014.json')
COCO_train = COCO(config.coco_ann_dir + 'instances_train2014.json')
ann_coco_file_val = config.coco_ann_dir + 'instances_val2014.json'
ann_coco_file_train = config.coco_ann_dir + 'instances_train2014.json'
with open(ann_coco_file_val) as f:
    ann_coco_1 = json.load(f)['categories']
coco_dict = {}
for i, det in enumerate(ann_coco_1):
    coco_dict[det['id']] = det['name']
    
    

In [6]:
class VQADataset_custom(Dataset):
    """VQA dataset"""

    def __init__(self, coco_pkl_file, ques_ann_path, ans_ann_path, mode):
        """
        Args:
            ques_ann (string): Path to the json file with ques_annotations.
            ans_ann (string): Path to the json file with ans_annotations.
        """
        self.coco_details = pickle.load(open(coco_pkl_file, 'rb'))['area_and_intersection']        
        self.questions = json.load(open(ques_ann_path, 'r'))['questions']  ###or self.questions = load_vocab(ques_ann_path)
        self.answers = json.load(open(ans_ann_path, 'r'))['answers']
        self.mode = mode

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        """Returns ONE data pair-image,match_coco_objects"""
        
        q = self.questions
        area_inter = self.coco_details
        ans = self.answers
        
        assert q[idx]['image_id'] == area_inter[idx]['image_id'] == ans[idx]['image_id']
        
        
        classes_img = area_inter[idx]['classes_img']
        percent_area_per_catId_all_inst = area_inter[idx]['percent_area_per_catId_all_inst']
        percent_area_per_catId_max_inst = area_inter[idx]['percent_area_per_catId_max_inst']
        if_intersect_overlap_sq5 = area_inter[idx]['if_intersect_overlap_sq5'] # or "if_intersect_overlap_default"
        #if_intersect_overlap_default = area_inter[idx]['if_intersect_overlap_default']
        flipped_if_intersect_overlap_sq5 = area_inter[idx]['flipped_if_intersect_overlap_sq5']
        iou_mask_flipped_mask = area_inter[idx]['iou_mask_flipped_mask']
        
        # print('Reading image data')
        img_id = self.questions[idx]['image_id']
        # print(img_id)

        question = self.questions[idx]['question']
        question_id = self.questions[idx]['question_id']
        # nouns_q = questions[idx]['nouns_q']
        # nouns_q_coco_stuff = questions[idx]['nouns_q_COCO_stuff']

        # print('Reading nouns data')
        nouns_q_coco = self.questions[idx]['nouns_q_COCO']
        nouns_ans = self.answers[idx]['ans_match_COCO']
        # print(img_id, nouns_img, nouns_q_coco, nouns_ans )
        
        answers = [i['answer'] for i in self.answers[idx]['answers']]
        
        question_type = self.answers[idx]['question_type']
        #multipe_choice_answer = self.answers[idx]['multipe_choice_answer']
        question_id2 = self.answers[idx]['question_id']
        assert question_id==question_id2
        
        answer_type = self.answers[idx]['answer_type']
        
        if cosider_flip_too:
            return answers, classes_img, nouns_q_coco, nouns_ans, img_id, question_id, question, \
                percent_area_per_catId_all_inst,if_intersect_overlap_sq5, percent_area_per_catId_max_inst,\
                flipped_if_intersect_overlap_sq5, iou_mask_flipped_mask, question_type, answer_type
        else:
            return answers, classes_img, nouns_q_coco, nouns_ans, img_id, question_id, question, \
                percent_area_per_catId_all_inst,if_intersect_overlap_sq5, percent_area_per_catId_max_inst, \
                question_type, answer_type
                

In [8]:
def count_dict(given_list):
    nouns_dict = {}
    for i in given_list:
        if i in nouns_dict:
            nouns_dict[i]+=1
        else:
            nouns_dict[i] =1
    return nouns_dict



In [12]:
mode = "train2014"   
coco = COCO_train
images_json_path ='coco_classes_' + mode + '_images.json'         #### contains img ids and classes in each image
question_path = './../iv_vqa_generation/tagged_' + mode + '_questions.json'        ## corresponds to question.json file
answer_path = './../iv_vqa_generation/tagged_'  + mode + '_answers.json'        
start = time.time()
dataset= VQADataset_custom(coco_train_pkl, question_path, answer_path, mode)

###GENERATING JSON FILE FOR COCO IMAGES!!
count_data_human_supervision = []
id_area_overlap = []
count=0
number_of_objects =0
allowed_char_in_ans = [str(i) for i in range(10)] #['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for i in range(len(dataset)):#range(len(dataset)):
    if cosider_flip_too:
        answers, classes_img, nouns_q, nouns_ans, img_id, ques_id, question,\
        percent_area_per_catId_all_inst,if_intersect_overlap, percent_area_per_catId_max_inst, \
        flipped_if_intersect_overlap_sq5, iou_mask_flipped_mask, question_type, answer_type = dataset[i]
    else:
        answers, classes_img, nouns_q, nouns_ans, img_id, ques_id, question,\
        percent_area_per_catId_all_inst,if_intersect_overlap, percent_area_per_catId_max_inst, \
        question_type, answer_type = dataset[i]     

    classes_img_set = sorted(list(set(classes_img)))

    nouns_q_str = [coco_dict[i] for i in nouns_q]
    classes_img_str = [coco_dict[i] for i in classes_img]

    count_nouns_q = count_dict(nouns_q)
    count_classes_img = count_dict(classes_img)

    ans_cond = [True if i in allowed_char_in_ans else False for i in answers[0]]
    final_ans_cond = True
    if False in ans_cond:
        final_ans_cond = False
    #ques_condition_neg ='number of' not in question and 'many' not in question and 'what number is' not in question\
    #and "What's the number" not in question and  'What number is' not in question
    if 'many' in question:
        if 'person' in count_nouns_q.keys():
            count_nouns_q['person']= count_nouns_q['person']-1
            if count_nouns_q['person']==0:
                del count_nouns_q['person']
        elif 1 in count_nouns_q.keys():
            count_nouns_q[1]= count_nouns_q[1]-1
            if count_nouns_q[1]==0:
                del count_nouns_q[1]

    ques_condition= 'many' in question or 'number of' in question 
    if len(set(answers))==1 and answer_type =='number' and final_ans_cond and ques_condition:

        ideal_candidates = []
        final_candi = []

        for key in count_classes_img:
            if key in count_nouns_q:
                ideal_candidates.append(key)
        for key in ideal_candidates: #count_classes_img:
            if count_classes_img[key]==int(answers[0]):
                final_candi.append(key)
        if len(final_candi)==1:
            count += 1
            cat_id = final_candi[0]
            num_obj_rem = int(answers[0])
            area_img = coco.imgs[img_id]['height'] * coco.imgs[img_id]['width']
            all_possible_ann = coco.loadAnns(coco.getAnnIds(img_id, cat_id))
            
            for ann in all_possible_ann:
                mini_dict = {}
                mini_dict['id'] = ann['id']
                area_occupied = round((ann['area']/area_img),3)
                mini_dict['area_occupied'] = area_occupied
                mini_dict['image_id'] = img_id
                mini_dict['question_id']= ques_id
                
                overlap_score = 0
        
                if area_occupied < 0.1:         
                    
                    mask_instance = coco.annToMask(ann)
                    mask_instance_dilated = scipy.ndimage.morphology.binary_dilation(mask_instance, square_dil)
                    all_possible_ann_mutable = coco.loadAnns(coco.getAnnIds(img_id, cat_id))
                    ann_index_in_all_possible_ann = all_possible_ann.index(ann)                
                    #print(all_possible_ann)
                    #print(all_possible_ann_mutable)
                    del all_possible_ann_mutable[ann_index_in_all_possible_ann]
                    #print(all_possible_ann)
                    #print(all_possible_ann_mutable)

                    if len(all_possible_ann_mutable) >0:      
                        maskTotal = np.zeros((coco.imgs[img_id]['height'], coco.imgs[img_id]['width']))

                        for ann2 in all_possible_ann_mutable:
                            cm = coco.annToMask(ann2)
                            maskTotal[:cm.shape[0], :cm.shape[1]] += cm                                 
                        maskTotal_dilated = scipy.ndimage.morphology.binary_dilation(maskTotal, square_dil)  

                        #print(img_id)
                        #plt.imshow(mask_instance_dilated, cmap='gray', interpolation='nearest'); plt.show()
                        #plt.imshow(maskTotal_dilated, cmap='gray', interpolation='nearest'); plt.show()

                        overlap_score = round(np.sum(mask_instance_dilated * maskTotal_dilated) / (np.sum(maskTotal_dilated)),3)

                        mini_dict['overlap_score_with_rest'] = overlap_score
                        id_area_overlap.append(mini_dict)  
                    else:
                        mini_dict['overlap_score_with_rest'] = 'only_candidate_instance'
                        
                else:
                    mini_dict['overlap_score_with_rest'] = 'area_instance >10%'                     
                id_area_overlap.append(mini_dict)                    
                #print(mini_dict)

with open(mode + 'coco_counting_id_area_overlap_only_one_considered_at_a_time.pickle', 'wb') as f:  
    pickle.dump(id_area_overlap,f,pickle.HIGHEST_PROTOCOL)
print('done')


done


In [15]:
with open(mode+ 'coco_counting_id_area_overlap_only_one_considered_at_a_time.pickle', 'rb') as file:
    id_area_overlap = pickle.load(file)
instance_ids_target = []
all_imgs_counting = []
target_count = 0
for mini_details in id_area_overlap:
    all_imgs_counting.append(mini_details['image_id'])
    overlap_score = mini_details['overlap_score_with_rest']
    if isinstance(overlap_score, float) or overlap_score == 'only_candidate_instance':
        if isinstance(overlap_score, float) and overlap_score<=0.0:
            #print(mini_details)
            target_count +=1
            instance_ids_target.append(mini_details['id'])
        else:
            target_count +=1
            instance_ids_target.append(mini_details['id'])            
print('so new edited #IQA:', target_count) 
print('unique images in counting edited set:', len(set(instance_ids_target)))
print('#counting IQA in original set:', len(id_area_overlap))
print('unique images in counting original set:', len(set(all_imgs_counting)))


In [ ]:
#Repeating for val2014 set

mode = "val2014"   
coco = COCO_val
images_json_path ='coco_classes_' + mode + '_images.json'         #### contains img ids and classes in each image
question_path = './../iv_vqa_generation/tagged_' + mode + '_questions.json'        ## corresponds to question.json file
answer_path = './../iv_vqa_generation/tagged_'  + mode + '_answers.json'        
start = time.time()
dataset= VQADataset_custom(coco_val_pkl, question_path, answer_path, mode)

###GENERATING JSON FILE FOR COCO IMAGES!!
count_data_human_supervision = []
id_area_overlap = []
count=0
number_of_objects =0
allowed_char_in_ans = [str(i) for i in range(10)] #['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for i in range(len(dataset)):#range(len(dataset)):
    if cosider_flip_too:
        answers, classes_img, nouns_q, nouns_ans, img_id, ques_id, question,\
        percent_area_per_catId_all_inst,if_intersect_overlap, percent_area_per_catId_max_inst, \
        flipped_if_intersect_overlap_sq5, iou_mask_flipped_mask, question_type, answer_type = dataset[i]
    else:
        answers, classes_img, nouns_q, nouns_ans, img_id, ques_id, question,\
        percent_area_per_catId_all_inst,if_intersect_overlap, percent_area_per_catId_max_inst, \
        question_type, answer_type = dataset[i]     

    classes_img_set = sorted(list(set(classes_img)))

    nouns_q_str = [coco_dict[i] for i in nouns_q]
    classes_img_str = [coco_dict[i] for i in classes_img]

    count_nouns_q = count_dict(nouns_q)
    count_classes_img = count_dict(classes_img)

    ans_cond = [True if i in allowed_char_in_ans else False for i in answers[0]]
    final_ans_cond = True
    if False in ans_cond:
        final_ans_cond = False
    #ques_condition_neg ='number of' not in question and 'many' not in question and 'what number is' not in question\
    #and "What's the number" not in question and  'What number is' not in question
    if 'many' in question:
        if 'person' in count_nouns_q.keys():
            count_nouns_q['person']= count_nouns_q['person']-1
            if count_nouns_q['person']==0:
                del count_nouns_q['person']
        elif 1 in count_nouns_q.keys():
            count_nouns_q[1]= count_nouns_q[1]-1
            if count_nouns_q[1]==0:
                del count_nouns_q[1]

    ques_condition= 'many' in question or 'number of' in question 
    if len(set(answers))==1 and answer_type =='number' and final_ans_cond and ques_condition:

        ideal_candidates = []
        final_candi = []

        for key in count_classes_img:
            if key in count_nouns_q:
                ideal_candidates.append(key)
        for key in ideal_candidates: #count_classes_img:
            if count_classes_img[key]==int(answers[0]):
                final_candi.append(key)
        if len(final_candi)==1:
            count += 1
            cat_id = final_candi[0]
            num_obj_rem = int(answers[0])
            area_img = coco.imgs[img_id]['height'] * coco.imgs[img_id]['width']
            all_possible_ann = coco.loadAnns(coco.getAnnIds(img_id, cat_id))
            
            for ann in all_possible_ann:
                mini_dict = {}
                mini_dict['id'] = ann['id']
                area_occupied = round((ann['area']/area_img),3)
                mini_dict['area_occupied'] = area_occupied
                mini_dict['image_id'] = img_id
                mini_dict['question_id']= ques_id
                
                overlap_score = 0
        
                if area_occupied < 0.1:         
                    
                    mask_instance = coco.annToMask(ann)
                    mask_instance_dilated = scipy.ndimage.morphology.binary_dilation(mask_instance, square_dil)
                    all_possible_ann_mutable = coco.loadAnns(coco.getAnnIds(img_id, cat_id))
                    ann_index_in_all_possible_ann = all_possible_ann.index(ann)                
                    #print(all_possible_ann)
                    #print(all_possible_ann_mutable)
                    del all_possible_ann_mutable[ann_index_in_all_possible_ann]
                    #print(all_possible_ann)
                    #print(all_possible_ann_mutable)

                    if len(all_possible_ann_mutable) >0:      
                        maskTotal = np.zeros((coco.imgs[img_id]['height'], coco.imgs[img_id]['width']))

                        for ann2 in all_possible_ann_mutable:
                            cm = coco.annToMask(ann2)
                            maskTotal[:cm.shape[0], :cm.shape[1]] += cm                                 
                        maskTotal_dilated = scipy.ndimage.morphology.binary_dilation(maskTotal, square_dil)  

                        #print(img_id)
                        #plt.imshow(mask_instance_dilated, cmap='gray', interpolation='nearest'); plt.show()
                        #plt.imshow(maskTotal_dilated, cmap='gray', interpolation='nearest'); plt.show()

                        overlap_score = round(np.sum(mask_instance_dilated * maskTotal_dilated) / (np.sum(maskTotal_dilated)),3)

                        mini_dict['overlap_score_with_rest'] = overlap_score
                        id_area_overlap.append(mini_dict)  
                    else:
                        mini_dict['overlap_score_with_rest'] = 'only_candidate_instance'
                        
                else:
                    mini_dict['overlap_score_with_rest'] = 'area_instance >10%'                     
                id_area_overlap.append(mini_dict)                    
                #print(mini_dict)

with open(mode + 'coco_counting_id_area_overlap_only_one_considered_at_a_time.pickle', 'wb') as f:  
    pickle.dump(id_area_overlap,f,pickle.HIGHEST_PROTOCOL)
print('done')
